In [19]:
import torch
from copy import deepcopy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [20]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset('trec')
#display(dataset)
dataset = dataset.remove_columns("label-fine")
#display(dataset)
dataset = dataset.rename_column("label-coarse", "label")
display(dataset)


train_validation_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
train_ds = Dataset.from_dict(train_validation_ds[0:4500])
validation_ds = Dataset.from_dict(train_validation_ds[4500:5000])
test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(500))])

display(train_ds)
display(validation_ds)
display(test_ds)

data = test_ds





Using custom data configuration default
Reusing dataset trec (C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 674.98it/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-bc761497303cb1ab.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-d432b7eeb4042bcd.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

In [21]:
display(data)
display(data[0:5])

Dataset({
    features: ['label', 'text'],
    num_rows: 500
})

{'label': [4, 0, 1, 4, 1],
 'text': ['What is the population of Venezuela ?',
  'What does target heart rate mean ?',
  'What currency does Luxembourg use ?',
  'How long did Rip Van Winkle sleep ?',
  'Material called linen is made from what plant ?']}

In [22]:
import textattack
dataset = textattack.datasets.HuggingFaceDataset(data)
display(dataset)

In [23]:
#attack the original model using TextFooler
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model_path = r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\models\CheckList\run3'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=6)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)



In [24]:
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019
attack = TextFoolerJin2019.build(model_wrapper)
attack.cuda_()
display(attack)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

In [25]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=-1, random_seed = 765,
                         log_to_csv = r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\results\CheckList\text_fooler_attack_results\run3\results.csv', 
                         csv_coloring_style = 'file')

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


textattack: Logging to CSV at path C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\results\CheckList\text_fooler_attack_results\run3\results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|▏                                                                                 | 1/500 [00:06<52:36,  6.33s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 1/500 [00:06<52:43,  6.34s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Num (99%)]] --> [[Loc (99%)]]

What is the [[population]] of Venezuela ?

What is the [[pueblo]] of Venezuela ?




[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 2/500 [00:07<30:33,  3.68s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                            | 2/500 [00:07<30:43,  3.70s/it]

...

--------------------------------------------- Result 498 ---------------------------------------------
[[Enty (99%)]] --> [[Hum (99%)]]

What was FDR 's [[dog]] 's name ?

What was FDR 's [[stacy]] 's name ?




C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 326 / 157 / 16 / 499: 100%|██████████████████▉| 499/500 [17:17<00:02,  2.08s/it]

--------------------------------------------- Result 499 ---------------------------------------------
[[Enty (98%)]] --> [[Loc (98%)]]

What is New York 's state [[bird]] ?

What is New York 's state [[corral]] ?




[Succeeded / Failed / Skipped / Total] 326 / 157 / 16 / 499: 100%|███████████████████| 500/500 [17:19<00:00,  2.08s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 327 / 157 / 16 / 500: 100%|███████████████████| 500/500 [17:19<00:00,  2.08s/it]

--------------------------------------------- Result 500 ---------------------------------------------
[[Loc (99%)]] --> [[Num (99%)]]

[[Where]] is the Holland Tunnel ?

[[When]] is the Holland Tunnel ?



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 327    |
| Number of failed attacks:     | 157    |
| Number of skipped attacks:    | 16     |
| Original accuracy:            | 96.8%  |
| Accuracy under attack:        | 31.4%  |
| Attack success rate:          | 67.56% |
| Average perturbed word %:     | 17.75% |
| Average num. words per input: | 6.5    |
| Avg num queries:              | 37.76  |
+-------------------------------+--------+


In [26]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))
results_html = logger.df[['original_text', 'perturbed_text']].to_html(escape=False)
path = r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\results\CheckList\text_fooler_attack_results\run3\results.html'
text_file = open(path, "w")
text_file.write(results_html.encode("gbk", 'ignore').decode("gbk", "ignore"))
text_file.close()
logger.flush()

textattack: Logging to CSV at path results.csv
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated 

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,original_text,perturbed_text
0,What is the population of Venezuela ?,What is the pueblo of Venezuela ?
1,What does target heart rate mean ?,What does target heart rate averages ?
2,What currency does Luxembourg use ?,What interchange does Luxembourg resort ?
3,How long did Rip Van Winkle sleep ?,How anymore did Rip Van Winkle sleep ?
4,Material called linen is made from what plant ?,Material called linen is reached from what centre ?
5,What is phenylalanine ?,What is phenylalanine ?
6,What is a group of turkeys called ?,What is a group of turkeys invited ?
7,What are enzymes ?,What are digested ?
8,Why is the sun yellow ?,Where is the sun yellow ?
9,What state is the geographic center of the lower 48 states ?,Wher sate is the geographic plants of the lower 48 states ?


In [27]:
import textattack

attack_log_manager = textattack.loggers.AttackLogManager()


for result in attack_results:
    attack_log_manager.log_result(result)

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()

attack_log_manager.flush()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 327    |
| Number of failed attacks:     | 157    |
| Number of skipped attacks:    | 16     |
| Original accuracy:            | 96.8%  |
| Accuracy under attack:        | 31.4%  |
| Attack success rate:          | 67.56% |
| Average perturbed word %:     | 17.75% |
| Average num. words per input: | 6.5    |
| Avg num queries:              | 37.76  |
+-------------------------------+--------+